valuation methods
fundamental metrics, PE, DE
analyst price price predictions


| # | Model                              | Input Frequency     | Output                |
| - | ---------------------------------- | ------------------- | --------------------- |
| 1 | **Multi-Stage DCF**                | Quarterly (TTM FCF) | Fair price per share  |
| 2 | **Adjusted Present Value (APV)**   | Quarterly           | Fair price per share  |
| 3 | **FCFE Model**                     | Quarterly           | Fair price per share  |
| 4 | **Dividend Discount (Gordon)**     | Quarterly           | Fair price per share  |
| 5 | **Residual Income (RI)**           | Quarterly           | Fair price per share  |
| 6 | **EVA-Based Valuation**            | Quarterly           | Fair price per share  |
| 7 | **Inverted Fama–French 5 Factor**  | Daily factors       | Required return $k_i$ |
| 8 | **Inverted q-Factor (Hou et al.)** | Daily factors       | Required return $k_i$ |


2. End-to-End Pipeline
Data Ingestion

Prices & factors: Daily price, FF5 & q-factor premia, T-bill rate

Fundamentals: Quarterly filings → compute TTM FCF, EPS, Book Value, NOPAT, Capital, Dividends, Share Count

Model Calculations (Quarterly Update)

DCF/APV/FCFE/Dividend/RI/EVA: Use latest TTM inputs + consensus or 5-yr historical growth (“g”) + dynamically estimated WACC (from yield curves + credit spreads)

Output: 6 fair-price estimates per stock

Expected Return Estimation (Daily)

Factor betas: Pre-estimate via 60-month rolling regressions

Required return
𝑘
𝑖
k
i
​
 :

𝑘
𝑖
FF5
=
𝑟
𝑓
+
∑
𝛽
𝑗

FF5
𝑗
,
𝑘
𝑖
𝑞
=
𝑟
𝑓
+
∑
𝛽
𝑗

q-factor
𝑗
k
i
FF5
​
 =r
f
​
 +∑β
j
​
 FF5
j
​
 ,k
i
q
​
 =r
f
​
 +∑β
j
​
 q-factor
j
​

Translate to “price” via
𝑃
=
EPS or
𝐷
1
𝑘
𝑖
−
𝑔
P=
k
i
​
 −g
EPS or D
1
​

​
  if desired, but here we’ll invert price→return directly.

Signal Aggregation & Ranking

Per-stock composite score:

𝑆
𝑖
=
1
𝑁
models
∑
𝑚
𝑟
^
𝑖
,
𝑚
S
i
​
 =
N
models
​

1
​
  
m
∑
​
  
r
^
  
i,m
​

Cross‐sectional rank of 𝑆𝑖S i​ → normalized to [–1, +1]

Idee om eerst alles apart te testen om soort feature importance te bepalen

In [2]:
#  API key 83S72RH62BUBUF7W

# 1 Data collection

In [3]:
#83S72RH62BUBUF7W

SyntaxError: invalid decimal literal (ipython-input-3098216842.py, line 1)

In [ ]:
company overview

In [ ]:
income statement

In [ ]:
balance sheet

In [ ]:
cash flow

In [ ]:
earnings history

In [ ]:
earnings estimates

In [6]:
import requests

# Replace with your actual API key
API_KEY = "83S72RH62BUBUF7W"
SYMBOL = "AAPL"

def fetch_alpha_vantage_data(function, symbol, api_key):
    """Fetches data from Alpha Vantage API."""
    base_url = "https://www.alphavantage.co/query"
    params = {
        "function": function,
        "symbol": symbol,
        "apikey": api_key,
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Fetch Company Overview
company_overview = fetch_alpha_vantage_data("OVERVIEW", SYMBOL, API_KEY)
print("Company Overview:")
display(company_overview)

Company Overview:


{'Symbol': 'AAPL',
 'AssetType': 'Common Stock',
 'Name': 'Apple Inc',
 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.",
 'CIK': '320193',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Industry': 'ELECTRONIC COMPUTERS',
 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US',
 'OfficialSite': 'https://www.apple.com',
 'FiscalYearEnd': 'September',
 'LatestQuarter': '2025-06-30',
 'MarketCapitalization': '3265333035000',
 'EBITDA': '141696008000',
 'PERatio

In [7]:
company_overview

{'Symbol': 'AAPL',
 'AssetType': 'Common Stock',
 'Name': 'Apple Inc',
 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.",
 'CIK': '320193',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Industry': 'ELECTRONIC COMPUTERS',
 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US',
 'OfficialSite': 'https://www.apple.com',
 'FiscalYearEnd': 'September',
 'LatestQuarter': '2025-06-30',
 'MarketCapitalization': '3265333035000',
 'EBITDA': '141696008000',
 'PERatio

In [8]:
# Fetch Income Statement
income_statement = fetch_alpha_vantage_data("INCOME_STATEMENT", SYMBOL, API_KEY)
print("\nIncome Statement:")
display(income_statement)


Income Statement:


{'symbol': 'AAPL',
 'annualReports': [{'fiscalDateEnding': '2024-09-30',
   'reportedCurrency': 'USD',
   'grossProfit': '180683000000',
   'totalRevenue': '391035000000',
   'costOfRevenue': '210352000000',
   'costofGoodsAndServicesSold': '210352000000',
   'operatingIncome': '123216000000',
   'sellingGeneralAndAdministrative': '26097000000',
   'researchAndDevelopment': '31370000000',
   'operatingExpenses': '57467000000',
   'investmentIncomeNet': 'None',
   'netInterestIncome': '0',
   'interestIncome': '0',
   'interestExpense': '0',
   'nonInterestIncome': 'None',
   'otherNonOperatingIncome': 'None',
   'depreciation': 'None',
   'depreciationAndAmortization': '11445000000',
   'incomeBeforeTax': '123485000000',
   'incomeTaxExpense': '29749000000',
   'interestAndDebtExpense': 'None',
   'netIncomeFromContinuingOperations': '93736000000',
   'comprehensiveIncomeNetOfTax': 'None',
   'ebit': '123216000000',
   'ebitda': '134661000000',
   'netIncome': '93736000000'},
  {'fisca

In [9]:
# Fetch Balance Sheet
balance_sheet = fetch_alpha_vantage_data("BALANCE_SHEET", SYMBOL, API_KEY)
print("\nBalance Sheet:")
display(balance_sheet)


Balance Sheet:


{'symbol': 'AAPL',
 'annualReports': [{'fiscalDateEnding': '2024-09-30',
   'reportedCurrency': 'USD',
   'totalAssets': '364980000000',
   'totalCurrentAssets': '152987000000',
   'cashAndCashEquivalentsAtCarryingValue': '29943000000',
   'cashAndShortTermInvestments': '29943000000',
   'inventory': '7286000000',
   'currentNetReceivables': '66243000000',
   'totalNonCurrentAssets': '211993000000',
   'propertyPlantEquipment': '55914000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'intangibleAssets': 'None',
   'intangibleAssetsExcludingGoodwill': 'None',
   'goodwill': 'None',
   'investments': 'None',
   'longTermInvestments': '91479000000',
   'shortTermInvestments': '35228000000',
   'otherCurrentAssets': '14287000000',
   'otherNonCurrentAssets': 'None',
   'totalLiabilities': '308030000000',
   'totalCurrentLiabilities': '176392000000',
   'currentAccountsPayable': '68960000000',
   'deferredRevenue': 'None',
   'currentDebt': 'None',
   'shortTermDebt': '2251100

In [10]:
# Fetch Cash Flow
cash_flow = fetch_alpha_vantage_data("CASH_FLOW", SYMBOL, API_KEY)
print("\nCash Flow:")
display(cash_flow)


Cash Flow:


{'symbol': 'AAPL',
 'annualReports': [{'fiscalDateEnding': '2024-09-30',
   'reportedCurrency': 'USD',
   'operatingCashflow': '118254000000',
   'paymentsForOperatingActivities': 'None',
   'proceedsFromOperatingActivities': 'None',
   'changeInOperatingLiabilities': 'None',
   'changeInOperatingAssets': 'None',
   'depreciationDepletionAndAmortization': '11445000000',
   'capitalExpenditures': '9447000000',
   'changeInReceivables': 'None',
   'changeInInventory': '-1046000000',
   'profitLoss': 'None',
   'cashflowFromInvestment': '2935000000',
   'cashflowFromFinancing': '-121983000000',
   'proceedsFromRepaymentsOfShortTermDebt': 'None',
   'paymentsForRepurchaseOfCommonStock': 'None',
   'paymentsForRepurchaseOfEquity': 'None',
   'paymentsForRepurchaseOfPreferredStock': 'None',
   'dividendPayout': '15234000000',
   'dividendPayoutCommonStock': '15234000000',
   'dividendPayoutPreferredStock': 'None',
   'proceedsFromIssuanceOfCommonStock': 'None',
   'proceedsFromIssuanceOfLong

In [11]:
# Fetch Earnings History
earnings_history = fetch_alpha_vantage_data("EARNINGS", SYMBOL, API_KEY)
print("\nEarnings History:")
display(earnings_history)


Earnings History:


{'symbol': 'AAPL',
 'annualEarnings': [{'fiscalDateEnding': '2025-06-30', 'reportedEPS': '5.62'},
  {'fiscalDateEnding': '2024-09-30', 'reportedEPS': '6.08'},
  {'fiscalDateEnding': '2023-09-30', 'reportedEPS': '6.12'},
  {'fiscalDateEnding': '2022-09-30', 'reportedEPS': '6.11'},
  {'fiscalDateEnding': '2021-09-30', 'reportedEPS': '5.62'},
  {'fiscalDateEnding': '2020-09-30', 'reportedEPS': '3.27'},
  {'fiscalDateEnding': '2019-09-30', 'reportedEPS': '2.98'},
  {'fiscalDateEnding': '2018-09-30', 'reportedEPS': '2.97'},
  {'fiscalDateEnding': '2017-09-30', 'reportedEPS': '2.3'},
  {'fiscalDateEnding': '2016-09-30', 'reportedEPS': '2.0675'},
  {'fiscalDateEnding': '2015-09-30', 'reportedEPS': '2.3'},
  {'fiscalDateEnding': '2014-09-30', 'reportedEPS': '1.6075'},
  {'fiscalDateEnding': '2013-09-30', 'reportedEPS': '1.415'},
  {'fiscalDateEnding': '2012-09-30', 'reportedEPS': '1.5775'},
  {'fiscalDateEnding': '2011-09-30', 'reportedEPS': '0.9875'},
  {'fiscalDateEnding': '2010-09-30', 'rep

In [12]:
# Fetch Earnings Estimates (Alpha Vantage doesn't have a dedicated endpoint for estimates,
# you might need to look for alternative sources or use the earnings endpoint and analyze)
# For now, I'll just print a message.
print("\nEarnings Estimates: Alpha Vantage does not have a dedicated endpoint for earnings estimates.")


Earnings Estimates: Alpha Vantage does not have a dedicated endpoint for earnings estimates.


# Task
Combine the income statement, balance sheet, and cash flow annual reports for IBM into a single dataframe and display it.

## Extract annual reports

### Subtask:
Extract the list of annual reports from the `income_statement`, `balance_sheet`, and `cash_flow` dictionaries.


**Reasoning**:
Extract the annual reports lists from the fetched financial data dictionaries.



In [13]:
income_annual_reports = income_statement['annualReports']
balance_annual_reports = balance_sheet['annualReports']
cashflow_annual_reports = cash_flow['annualReports']

In [14]:
import pandas as pd

income_df = pd.DataFrame(income_annual_reports)
balance_df = pd.DataFrame(balance_annual_reports)
cashflow_df = pd.DataFrame(cashflow_annual_reports)

In [15]:
combined_df = pd.merge(income_df, balance_df, on='fiscalDateEnding', how='inner')
combined_df = pd.merge(combined_df, cashflow_df, on='fiscalDateEnding', how='inner')
display(combined_df)

,fiscalDateEnding,reportedCurrency_x,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome_y
0,2024-09-30,USD,180683000000,391035000000,210352000000,210352000000,123216000000,26097000000,31370000000,57467000000,...,15234000000,None,None,None,None,-94949000000,None,None,None,93736000000
1,2023-09-30,USD,169148000000,383285000000,214137000000,214137000000,114301000000,24932000000,29915000000,54847000000,...,15025000000,None,None,None,None,-77550000000,None,5760000000,None,96995000000
2,2022-09-30,USD,170782000000,394328000000,223546000000,223546000000,119437000000,25094000000,26251000000,51573000000,...,14841000000,None,None,None,None,-89402000000,None,-10952000000,None,99803000000
3,2021-09-30,USD,152836000000,365817000000,212981000000,212981000000,108949000000,21973000000,21914000000,43887000000,...,14467000000,None,None,None,None,-85971000000,None,-3860000000,None,94680000000
4,2020-09-30,USD,104956000000,274515000000,169559000000,169559000000,66288000000,19916000000,18752000000,38668000000,...,14081000000,None,None,None,None,-72358000000,None,-10435000000,None,57411000000
5,2019-09-30,USD,98392000000,260174000000,161782000000,161782000000,63930000000,18245000000,16217000000,34462000000,...,14119000000,None,None,None,None,-66897000000,None,24311000000,None,55256000000
6,2018-09-30,USD,101839000000,265595000000,163756000000,163756000000,70898000000,16705000000,14236000000,30941000000,...,13712000000,None,None,None,None,-72738000000,None,5624000000,None,59531000000
7,2017-09-30,USD,88186000000,229234000000,141048000000,141048000000,61344000000,15261000000,11581000000,26842000000,...,12769000000,None,None,None,None,-32900000000,None,-195000000,None,48351000000
8,2016-09-30,USD,84263000000,215639000000,131376000000,131376000000,60024000000,14194000000,10045000000,24239000000,...,12150000000,None,None,None,None,-29722000000,None,-636000000,None,45687000000
9,2015-09-30,USD,93626000000,233715000000,140089000000,140089000000,71230000000,14329000000,8067000000,22396000000,...,11561000000,None,None,None,None,-35253000000,None,7276000000,None,53394000000


# **ratio's**

**📊 Liquidity Ratios
Measure a company’s ability to meet short-term obligations. **

Current Ratio	Current Assets / Current Liabilities	Measures short-term solvency.

Quick Ratio (Acid-Test)	(Current Assets - Inventory) / Current Liabilities	Measures liquidity without relying on inventory.

Cash Ratio	(Cash + Cash Equivalents) / Current Liabilities	Most conservative liquidity ratio.


**💰 Profitability Ratios
Evaluate a company’s ability to generate profit. **



Gross Profit Margin	Gross Profit / Revenue × 100	Shows profitability after cost of goods sold.

Operating Profit Margin	Operating Profit / Revenue × 100	Measures core business profitability.

Net Profit Margin	Net Income / Revenue × 100	Final profitability after all expenses.

Return on Assets (ROA)	Net Income / Total Assets × 100	Efficiency in using assets to generate profit.

Return on Equity (ROE)	Net Income / Shareholders’ Equity × 100	Measures return on shareholder investment.

Return on Capital Employed (ROCE)	EBIT / Capital Employed × 100	Efficiency of capital use.

⚖️ Solvency (Leverage) Ratios
Assess long-term financial stability.

Debt-to-Equity Ratio	Total Debt / Total Equity	Measures financial leverage.

Equity Ratio	Equity / Total Assets × 100	Indicates the proportion of assets financed by equity.

Interest Coverage Ratio	EBIT / Interest Expense	Shows how easily interest payments are covered.

Debt Ratio	Total Liabilities / Total Assets	Proportion of assets financed by debt.

📈 Efficiency (Activity) Ratios
Measure how well assets and liabilities are managed.

Inventory Turnover	Cost of Goods Sold / Average Inventory	Measures inventory efficiency.

Accounts Receivable Turnover	Net Credit Sales / Average Accounts Receivable	Shows how quickly receivables are collected.

Accounts Payable Turnover	Cost of Goods Sold / Average Accounts Payable	Efficiency in paying suppliers.

Asset Turnover Ratio	Revenue / Average Total
Assets	Shows how effectively assets generate sales.

Working Capital Turnover	Revenue / Average Working Capital	Efficiency in using working capital.


**💵 Market Ratios (Valuation Ratios)
Used to assess stock market performance and investor returns.**


Earnings Per Share (EPS)	(Net Income - Preferred Dividends) / Weighted Avg. Shares	Shows profitability per share.

Price-to-Earnings (P/E)	Market Price per Share / EPS	Indicates stock valuation.

Price-to-Book (P/B)	Market Price per Share / Book Value per Share	Compares market and book value.

Dividend Yield	Annual Dividends per Share / Price per Share × 100	Measures dividend income vs stock price.

Dividend Payout Ratio	Dividends / Net Income × 100	Proportion of earnings paid to shareholders.

Cash flow models

over or undervaluation